In [1]:
pip install python-binance

     |████████████████████████████████| 3.1 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 362 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 12.2 MB/s eta 0:00:01
  Using cached incremental-17.5.0-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 48 kB 15.5 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 2.5 MB/s  eta 0:00:01
  Created wheel for Twisted: filename=Twisted-20.3.0-cp38-cp38-macosx_10_9_x86_64.whl size=3050323 sha256=66303cf6c41ddcd72a1aeb1cccff8bc351b852c3cd4c098ca224995c19607f58
  Stored in directory: /Users/thunchanok/Library/Caches/pip/wheels/f2/36/1b/99fe6d339e1559e421556c69ad7bc8c869145e86a756c403f4
Successfully built Twisted
Note: you may need to restart the kernel to use updated packages.


# combine without print

In [56]:
# https://python-binance.readthedocs.io/en/latest/
from binance.client import Client
import datetime
import pandas as pd
import math
import numpy as np

client = Client('dyZMalaTBM0vNWwLKeDfjp6r7CxswhmqHGb29LIA15lvR1klcmUp0qmTxQueKMGD', 'AnYFvyhK1dTntOiiq2wrCXbEJms11bLFYWvjFcHfJMLXqSvGleiO3NsRzalsLQFL')

# get market depth
depth = client.get_order_book(symbol='ETHBTC')

t=(client.get_exchange_info())['serverTime']
time=datetime.datetime.fromtimestamp(t/1000)

depth
bid=depth['bids']
ask=depth['asks']

# BID-ASK DATAFRAME
df_bid = pd.DataFrame(columns=["price","amount"])
df_ask = pd.DataFrame(columns=["price","amount"])
for i in range(len(bid)):
    df_bid.loc[i]=bid[i]
for i in range(len(ask)):
    df_ask.loc[i]=ask[i]
df_ask=df_ask.sort_values(by='price', ascending=True)
df_bid=df_bid.sort_values(by='price', ascending=False)

# spread
max_bid=float(df_bid.iloc[0]['price'])
min_ask=float(df_ask.iloc[0]['price'])
spread=((min_ask-max_bid)/min_ask)*100


# SLIPPAGE
avg_buy_price=0
best_ask=min_ask
best_bid=max_bid
midPrice=(best_ask+best_bid)/2
sum_buy_price=0
btc_sell_list=[0.1,0.3,0.5,1,5]
shrimpy_slippage_list=[]
kaiko_slippage_list=[]
bybit_slippage_list=[]
bn_slippage_list=[]


for btc_sell_amount in btc_sell_list:
    sum_amount=0
    pi=float(df_ask.iloc[0]['price'])
    sum_buy_price=0 
    
    
    for i in range (len(ask)):
    
        if btc_sell_amount > sum_amount:
            sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
            sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
            if sum_amount>=btc_sell_amount:
                pf=float(df_ask.iloc[i]['price'])
                index=i+1
                
                break
    if sum_amount<btc_sell_amount and i==len(ask)-1:
        bn_slippage_list.append([-1,-1,-1])
        while(len(bn_slippage_list)!= len(btc_sell_list)) :
            bn_slippage_list.append([-1,-1,-1])

        break

    b_sum_amount=0
    sum_ask_price= 0
    b_pi=float(df_bid.iloc[0]['price'])
    for i in range (len(bid)):
        if btc_sell_amount > b_sum_amount:
            b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
            sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
            
            if b_sum_amount>=btc_sell_amount:
                b_pf=float(df_bid.iloc[i]['price'])
                b_index=i+1
                break

    if b_sum_amount<btc_sell_amount and i==len(ask)-1:
    
        break
    avg_buy_price=sum_buy_price/(index)

    bybit_slippage=math.fabs(b_pf-pf)/midPrice
    shrimpy_slippage=(math.fabs(pf-pi)/pi)
    kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

    bn_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])

bn_slp_rows=(np.array(bn_slippage_list).T).tolist()
bn_slippage_spread_rows=[time,spread,bn_slp_rows[0],bn_slp_rows[1],bn_slp_rows[2]]

bn_slippage_spread = pd.DataFrame(columns=["date","bn-spread","bn-slp-shrimpy","bn-slp-kaiko","bn-slp-bybit"])
bn_slippage_spread.to_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv", index=False)

df=pd.read_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv")
df.loc[len(df)]=bn_slippage_spread_rows
df.to_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv", index=False)
df


,date,bn-spread,bn-slp-shrimpy,bn-slp-kaiko,bn-slp-bybit
0,2020-09-30 14:47:05.957,0.003013,"[0.0, 0.0, 0.0, 0.0, 6.026637738811548e-05]","[1.506682135271429e-05, 1.506682135271429e-05,...","[3.013364270542858e-05, 3.013364270542858e-05,..."


# combine with print

In [55]:
# https://python-binance.readthedocs.io/en/latest/
from binance.client import Client
import datetime
import pandas as pd
import math
import numpy as np

client = Client('dyZMalaTBM0vNWwLKeDfjp6r7CxswhmqHGb29LIA15lvR1klcmUp0qmTxQueKMGD', 'AnYFvyhK1dTntOiiq2wrCXbEJms11bLFYWvjFcHfJMLXqSvGleiO3NsRzalsLQFL')

# get market depth
depth = client.get_order_book(symbol='ETHBTC')

t=(client.get_exchange_info())['serverTime']
time=datetime.datetime.fromtimestamp(t/1000)
print(time)
depth
bid=depth['bids']
ask=depth['asks']

# BID-ASK DATAFRAME
df_bid = pd.DataFrame(columns=["price","amount"])
df_ask = pd.DataFrame(columns=["price","amount"])
for i in range(len(bid)):
    df_bid.loc[i]=bid[i]
for i in range(len(ask)):
    df_ask.loc[i]=ask[i]
df_ask=df_ask.sort_values(by='price', ascending=True)
df_bid=df_bid.sort_values(by='price', ascending=False)

# spread
max_bid=float(df_bid.iloc[0]['price'])
min_ask=float(df_ask.iloc[0]['price'])
spread=((min_ask-max_bid)/min_ask)*100


# SLIPPAGE
avg_buy_price=0
best_ask=min_ask
best_bid=max_bid
midPrice=(best_ask+best_bid)/2
sum_buy_price=0
btc_sell_list=[0.1,0.3,0.5,1,5]
shrimpy_slippage_list=[]
kaiko_slippage_list=[]
bybit_slippage_list=[]
bn_slippage_list=[]

print("  ")
print("best_ask "+str(best_ask))
print("best_bid "+str(best_bid))
print("midPrice "+str(midPrice))


for btc_sell_amount in btc_sell_list:
    sum_amount=0
    pi=float(df_ask.iloc[0]['price'])
    sum_buy_price=0 
    
    print("  ")
    print("___________________________________________________________________________________________________")
    print("ASK ")
    
    
    for i in range (len(ask)):
        print(" <= accumulative amount => "+str(sum_amount)+" ********** <= accumulative price => "+str(sum_buy_price))
        if btc_sell_amount > sum_amount:
            sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
            sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
            if sum_amount>=btc_sell_amount:
                pf=float(df_ask.iloc[i]['price'])
                index=i+1
                
                print("     (transaction order : "+str(i+1)+")")
                print("     (ask final price : "+str(pf)+")")
                
                
                break
    if sum_amount<btc_sell_amount and i==len(ask)-1:
        bn_slippage_list.append([-1,-1,-1])
        while(len(bn_slippage_list)!= len(btc_sell_list)) :
            bn_slippage_list.append([-1,-1,-1])
        
        print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
        print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
        print("___________________________________________________________________________________________________")
        print("")

        break
    print("BID ")
    b_sum_amount=0
    sum_ask_price= 0
    b_pi=float(df_bid.iloc[0]['price'])
    for i in range (len(bid)):
        if btc_sell_amount > b_sum_amount:
            b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
            sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
            print(" <= accumulative amount => "+str(b_sum_amount)+" ********** <= accumulative price => "+str(sum_ask_price))
            if b_sum_amount>=btc_sell_amount:
                b_pf=float(df_bid.iloc[i]['price'])
                b_index=i+1
                print("     (transaction order : "+str(i+1)+")")
                print("     (bid final price  : "+str(b_pf)+")")
                break

    if b_sum_amount<btc_sell_amount and i==len(ask)-1:
        print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
        print("___________________________________________________________________________________________________")
        print("")
        break
    avg_buy_price=sum_buy_price/(index)
    
    print("avg_buy_price "+str(avg_buy_price))

    bybit_slippage=math.fabs(b_pf-pf)/midPrice
    shrimpy_slippage=(math.fabs(pf-pi)/pi)
    kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

    bn_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])
    
    print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> "+str(shrimpy_slippage)+" ")
    print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> "+str(kaiko_slippage)+" ")
    print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> "+str(bybit_slippage)+" ")
    print("___________________________________________________________________________________________________")
    print("")


bn_slp_rows=(np.array(bn_slippage_list).T).tolist()
bn_slippage_spread_rows=[time,spread,bn_slp_rows[0],bn_slp_rows[1],bn_slp_rows[2]]

bn_slippage_spread = pd.DataFrame(columns=["date","bn-spread","bn-slp-shrimpy","bn-slp-kaiko","bn-slp-bybit"])
bn_slippage_spread.to_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv", index=False)

df=pd.read_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv")
df.loc[len(df)]=bn_slippage_spread_rows
df.to_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv", index=False)
df


2020-09-30 14:45:04.624000
  
best_ask 0.033188
best_bid 0.033187
midPrice 0.0331875
  
___________________________________________________________________________________________________
ASK 
 <= accumulative amount => 0 ********** <= accumulative price => 0
     (transaction order : 1)
     (ask final price : 0.033188)
BID 
 <= accumulative amount => 4.426 ********** <= accumulative price => 0.033187
     (transaction order : 1)
     (bid final price  : 0.033187)
avg_buy_price 0.033188
shrimpy_slippage of 0.1 BTC --> 0.0 
kaiko_slippage of 0.1 BTC --> 1.5065913371013184e-05 
bybit_slippage of 0.1 BTC --> 3.0131826742026367e-05 
___________________________________________________________________________________________________

  
___________________________________________________________________________________________________
ASK 
 <= accumulative amount => 0 ********** <= accumulative price => 0
     (transaction order : 1)
     (ask final price : 0.033188)
BID 
 <= accumulative a

,date,bn-spread,bn-slp-shrimpy,bn-slp-kaiko,bn-slp-bybit
0,2020-09-30 14:45:04.624,0.003013,"[0.0, 0.0, 0.0, 0.0, 0.0]","[1.5065913371013184e-05, 1.5065913371013184e-0...","[3.0131826742026367e-05, 3.0131826742026367e-0..."


# Separate

In [57]:
# https://python-binance.readthedocs.io/en/latest/
from binance.client import Client
import datetime
import pandas as pd
import math
import numpy as np

client = Client('dyZMalaTBM0vNWwLKeDfjp6r7CxswhmqHGb29LIA15lvR1klcmUp0qmTxQueKMGD', 'AnYFvyhK1dTntOiiq2wrCXbEJms11bLFYWvjFcHfJMLXqSvGleiO3NsRzalsLQFL')

# get market depth
depth = client.get_order_book(symbol='ETHBTC')

t=(client.get_exchange_info())['serverTime']
time=datetime.datetime.fromtimestamp(t/1000)
print(time)
depth
bid=depth['bids']
ask=depth['asks']

2020-09-30 14:48:11.362000


In [60]:
# BID-ASK DATAFRAME
import pandas as pd
df_bid = pd.DataFrame(columns=["price","amount"])
df_ask = pd.DataFrame(columns=["price","amount"])
for i in range(len(bid)):
    df_bid.loc[i]=bid[i]
for i in range(len(ask)):
    df_ask.loc[i]=ask[i]
df_ask=df_ask.sort_values(by='price', ascending=True)
df_bid=df_bid.sort_values(by='price', ascending=False)

print('ask')
print(df_ask)
print('bid')
print(df_bid)

ask
         price       amount
0   0.03318100  66.63700000
1   0.03318200  24.00000000
2   0.03318300   3.68400000
3   0.03318400  24.11400000
4   0.03318500   1.99900000
..         ...          ...
95  0.03329000  15.61600000
96  0.03329100   0.02800000
97  0.03329200   0.18100000
98  0.03329300   0.02300000
99  0.03329400   0.01200000

[100 rows x 2 columns]
bid
         price       amount
0   0.03318000  29.93400000
1   0.03317900   1.00000000
2   0.03317800   3.29200000
3   0.03317700   6.11200000
4   0.03317600   6.00000000
..         ...          ...
95  0.03303900  92.56200000
96  0.03303700   0.00400000
97  0.03303600   1.91600000
98  0.03303400   0.03600000
99  0.03303300   0.00900000

[100 rows x 2 columns]


In [43]:
# spread
# https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
# AL(lowest ask price) - BH(highest buy price) = Spread
# Percent Spread = (Spread / lowest ask price) x 100
max_bid=float(df_bid.iloc[0]['price'])
min_ask=float(df_ask.iloc[0]['price'])
spread=((min_ask-max_bid)/min_ask)*100

print('best bid '+str(max_bid))
print('best ask '+str(min_ask))
print('spread '+str(spread))


best bid 0.033182
best ask 0.033183
spread 0.0030135912967304383


In [45]:
# SLIPPAGE
import math
avg_buy_price=0
best_ask=min_ask
best_bid=max_bid
midPrice=(best_ask+best_bid)/2
sum_buy_price=0
btc_sell_list=[0.1,0.3,0.5,1,5]
shrimpy_slippage_list=[]
kaiko_slippage_list=[]
bybit_slippage_list=[]
bn_slippage_list=[]

print("  ")
print("best_ask "+str(best_ask))
print("best_bid "+str(best_bid))
print("midPrice "+str(midPrice))


for btc_sell_amount in btc_sell_list:
    sum_amount=0
    pi=float(df_ask.iloc[0]['price'])
    sum_buy_price=0 
    
    print("  ")
    print("___________________________________________________________________________________________________")
    print("ASK ")
    
    
    for i in range (len(ask)):
        print(" <= accumulative amount => "+str(sum_amount)+" ********** <= accumulative price => "+str(sum_buy_price))
        if btc_sell_amount > sum_amount:
            sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
            sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
            if sum_amount>=btc_sell_amount:
                pf=float(df_ask.iloc[i]['price'])
                index=i+1
                
                print("     (transaction order : "+str(i+1)+")")
                print("     (ask final price : "+str(pf)+")")
                
                
                break
    if sum_amount<btc_sell_amount and i==len(ask)-1:
        bn_slippage_list.append([-1,-1,-1])
        while(len(bn_slippage_list)!= len(btc_sell_list)) :
            bn_slippage_list.append([-1,-1,-1])
        
        print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
        print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
        print("___________________________________________________________________________________________________")
        print("")

        break
    print("BID ")
    b_sum_amount=0
    sum_ask_price= 0
    b_pi=float(df_bid.iloc[0]['price'])
    for i in range (len(bid)):
        if btc_sell_amount > b_sum_amount:
            b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
            sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
            print(" <= accumulative amount => "+str(b_sum_amount)+" ********** <= accumulative price => "+str(sum_ask_price))
            if b_sum_amount>=btc_sell_amount:
                b_pf=float(df_bid.iloc[i]['price'])
                b_index=i+1
                print("     (transaction order : "+str(i+1)+")")
                print("     (bid final price  : "+str(b_pf)+")")
                break

    if b_sum_amount<btc_sell_amount and i==len(ask)-1:
        print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
        print("___________________________________________________________________________________________________")
        print("")
        break
    avg_buy_price=sum_buy_price/(index)
    
    print("avg_buy_price "+str(avg_buy_price))

    bybit_slippage=math.fabs(b_pf-pf)/midPrice
    shrimpy_slippage=(math.fabs(pf-pi)/pi)
    kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

    bn_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])
    
    print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> "+str(shrimpy_slippage)+" ")
    print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> "+str(kaiko_slippage)+" ")
    print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> "+str(bybit_slippage)+" ")
    print("___________________________________________________________________________________________________")
    print("")


  
best_ask 0.033183
best_bid 0.033182
midPrice 0.033182500000000004
  
___________________________________________________________________________________________________
ASK 
 <= accumulative amount => 0 ********** <= accumulative price => 0
     (transaction order : 1)
     (ask final price : 0.033183)
BID 
 <= accumulative amount => 28.992 ********** <= accumulative price => 0.033182
     (transaction order : 1)
     (bid final price  : 0.033182)
avg_buy_price 0.033183
shrimpy_slippage of 0.1 BTC --> 0.0 
kaiko_slippage of 0.1 BTC --> 1.5068183530281356e-05 
bybit_slippage of 0.1 BTC --> 3.0136367060771824e-05 
___________________________________________________________________________________________________

  
___________________________________________________________________________________________________
ASK 
 <= accumulative amount => 0 ********** <= accumulative price => 0
     (transaction order : 1)
     (ask final price : 0.033183)
BID 
 <= accumulative amount => 28.992

In [54]:
import numpy as np
bn_slp_rows=(np.array(bn_slippage_list).T).tolist()
bn_slippage_spread_rows=[time,spread,bn_slp_rows[0],bn_slp_rows[1],bn_slp_rows[2]]

bn_slippage_spread = pd.DataFrame(columns=["date","bn-spread","bn-slp-shrimpy","bn-slp-kaiko","bn-slp-bybit"])
bn_slippage_spread.to_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv", index=False)

df=pd.read_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv")
df.loc[len(df)]=bn_slippage_spread_rows
df.to_csv("//Users//thunchanok//Downloads//bn_slippage_spread.csv", index=False)
df

,date,bn-spread,bn-slp-shrimpy,bn-slp-kaiko,bn-slp-bybit
0,2020-09-30 14:24:32.674,0.003014,"[0.0, 0.0, 0.0, 0.0, 0.0]","[1.5068183530281356e-05, 1.5068183530281356e-0...","[3.0136367060771824e-05, 3.0136367060771824e-0..."
